## Task Description

Author: Zhixin Song \
Date: 09/25/2021

- Prepare 4 random 4-qubit quantum states of your choice.
- Create and train a variational circuit that transforms input states into predefined output states. Namely
    - if random state 1 is provided, it returns state $|0011\rangle$
    - if random state 2 is provided, it returns state $|0101\rangle$
    - if random state 3 is provided, it returns state $|1010\rangle$
    - if random state 4 is provided, it returns state $|1100\rangle$
    
What would happen if you provided a different state?

Analyze and discuss the results.


## Solution -- The approach of Variational Compiling

This solution is inspired by: [Sharma, Kunal, et al. "Noise resilience of variational quantum compiling." *New Journal of Physics* 22.4 (2020): 043006.](https://arxiv.org/pdf/1908.04416.pdf)



Choose the inital 4 random 4-qubit quantum states as a subset of the computational basis:

$$
S = \bigg\{|1000\rangle, |0100\rangle, |0010\rangle, |0001\rangle \bigg\}.
\tag{1}
$$

Then, we only need to find and train a paramterized quantum circuit (PQC) to realize the partial transformation:

$$
U_p|0001\rangle \rightarrow |0011\rangle,~  
U_p|0010\rangle \rightarrow |0101\rangle,~
U_p|0100\rangle \rightarrow |1010\rangle,~ 
U_p|1000\rangle \rightarrow |1100\rangle. 
\tag{2}
$$

Or equivalently,
$$
U_p|1\rangle \rightarrow |3\rangle,~  
U_p|2\rangle \rightarrow |5\rangle,~
U_p|4\rangle \rightarrow |10\rangle,~ 
U_p|8\rangle \rightarrow |12\rangle. 
\tag{3}
$$

To make it a complete transformation, let all the other computational basis vectors unchanged.

$$
|1110\rangle \rightarrow |1110\rangle,~  
|0111\rangle \rightarrow |0111\rangle,~
|0000\rangle \rightarrow |0000\rangle,~ 
|1111\rangle \rightarrow |1111\rangle,~ \cdots
\tag{4}
$$

We can write this unitary transformation explicitly as:

$$
U = 
\begin{bmatrix}
1 &0 &0 &0 &0 &0 &0 &0 &0 &0 &0 &0 &0 &0 &0 &0\\
0 &0 &0 &1 &0 &0 &0 &0 &0 &0 &0 &0 &0 &0 &0 &0\\
0 &0 &0 &0 &0 &1 &0 &0 &0 &0 &0 &0 &0 &0 &0 &0\\
0 &1 &0 &0 &0 &0 &0 &0 &0 &0 &0 &0 &0 &0 &0 &0\\
0 &0 &0 &0 &0 &0 &0 &0 &0 &0 &1 &0 &0 &0 &0 &0\\
0 &0 &1 &0 &0 &0 &0 &0 &0 &0 &0 &0 &0 &0 &0 &0\\
0 &0 &0 &0 &0 &0 &1 &0 &0 &0 &0 &0 &0 &0 &0 &0\\
0 &0 &0 &0 &0 &0 &0 &1 &0 &0 &0 &0 &0 &0 &0 &0\\
0 &0 &0 &0 &0 &0 &0 &0 &0 &0 &0 &0 &1 &0 &0 &0\\
0 &0 &0 &0 &0 &0 &0 &0 &0 &1 &0 &0 &0 &0 &0 &0\\
0 &0 &0 &0 &1 &0 &0 &0 &0 &0 &0 &0 &0 &0 &0 &0\\
0 &0 &0 &0 &0 &0 &0 &0 &0 &0 &0 &1 &0 &0 &0 &0\\
0 &0 &0 &0 &0 &0 &0 &0 &1 &0 &0 &0 &0 &0 &0 &0\\
0 &0 &0 &0 &0 &0 &0 &0 &0 &0 &0 &0 &0 &1 &0 &0\\
0 &0 &0 &0 &0 &0 &0 &0 &0 &0 &0 &0 &0 &0 &1 &0\\
0 &0 &0 &0 &0 &0 &0 &0 &0 &0 &0 &0 &0 &0 &0 &1
\end{bmatrix}.
\tag{5}
$$

Now, the problem has been converted to how to use a PQC $U(\vec{\theta})$ to realize such a transformation.

## Implementation with Paddle Quantum

GitHub Link: https://github.com/PaddlePaddle/Quantum

In [4]:
from IPython.display import clear_output
!pip install paddle-quantum==2.1.2
clear_output()

In [2]:
import numpy as np
import paddle
from paddle_quantum.circuit import UAnsatz
from paddle_quantum.utils import dagger, trace_distance
from paddle_quantum.state import density_op_random

# Number of qubits
N = 4 

# The matrix form of U
U0 = paddle.to_tensor(np.matrix([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                                [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                                [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                                [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
                                [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                                [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                                [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
                                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
                                [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
                                [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
                                [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
                                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
                                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
                                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]],
                       dtype="float64"))

# Check this is a unitary matrix
print(U0.numpy()@dagger(U0).numpy() - np.eye(2**N))

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


### PQC 

In [10]:
# Constructing a parametried quantum circuit
def V_theta(theta, N, D):
    # Initialize the quantum circuit
    cir = UAnsatz(N)
    # Call the built-in template {Ry rotation on each qubit + adjacent CX gates}x Depth D
    cir.real_entangled_layer(theta[:D], D)

    return cir

### Loss function

Here, we use trace distance to evalute the training process:

$$
\mathcal{L}(\vec{\theta}) (U, V_\theta)= 1 - \frac{1}{d^2}\big|tr(UV_\theta^\dagger)\big|^2.
\tag{6}
$$

In [11]:
# Training loss function
class Net(paddle.nn.Layer):
    def __init__(self, shape, dtype="float64", ):
        super(Net, self).__init__()

        self.theta = self.create_parameter(shape=shape,
                                           default_initializer=paddle.nn.initializer.Uniform(low=0.0, high=2 * np.pi),
                                           dtype=dtype, is_bias=False)

    def forward(self, N, D):
        
        # Read out the matrix representation of PQC
        cir = V_theta(self.theta, N, D)
        V = cir.U
        
        # Define the loss function
        loss = 1 - (dagger(V).matmul(U0).trace().abs() / V.shape[0]) ** 2

        return loss, cir 

In [30]:
D = 40      # Set the depth of PQC
LR = 0.25   # Set the learning rate
ITR = 100   # Set the number of optimization iterations

# Determine shape of parameters
net = Net(shape=[D + 1, N, 1])
# Using Adam optimizer 
opt = paddle.optimizer.Adam(learning_rate=LR, parameters=net.parameters())

# Optimization loop
for itr in range(1, ITR + 1):
    loss, cir = net.forward(N, D)
    
    # Backpropagation with automatic differentiation
    loss.backward()
    opt.minimize(loss)
    opt.clear_grad()

    # Record Training process
    if itr % 20 == 0:
        print("iter:", itr, "loss:", "%.4f" % loss.numpy())

iter: 20 loss: 0.0604
iter: 40 loss: 0.0126
iter: 60 loss: 0.0135
iter: 80 loss: 0.0005
iter: 100 loss: 0.0035


In [55]:
U_compiled = np.around(cir.U.numpy().real, 1)
print(U_compiled)

[[ 1.  0. -0. -0.  0.  0.  0.  0. -0. -0.  0.  0. -0. -0. -0.  0.]
 [ 0.  0.  0.  1.  0.  0. -0.  0. -0. -0. -0. -0.  0. -0. -0.  0.]
 [-0. -0.  0. -0. -0.  1. -0.  0. -0. -0.  0. -0.  0.  0.  0.  0.]
 [-0.  1.  0. -0.  0.  0. -0. -0.  0.  0. -0. -0. -0.  0. -0.  0.]
 [-0.  0.  0.  0.  0. -0.  0.  0. -0. -0.  1.  0.  0.  0.  0. -0.]
 [ 0. -0.  1. -0. -0. -0.  0. -0.  0.  0. -0.  0.  0.  0. -0.  0.]
 [-0.  0. -0.  0.  0.  0.  1.  0.  0. -0. -0.  0.  0.  0. -0.  0.]
 [-0.  0.  0. -0. -0. -0. -0.  1.  0. -0. -0. -0.  0.  0.  0.  0.]
 [ 0.  0. -0. -0. -0. -0. -0. -0. -0. -0. -0.  0.  1.  0.  0. -0.]
 [ 0. -0. -0.  0. -0.  0.  0.  0. -0.  1.  0. -0.  0. -0.  0. -0.]
 [-0. -0.  0. -0.  1.  0. -0.  0.  0.  0. -0.  0.  0.  0. -0. -0.]
 [-0.  0. -0.  0. -0.  0. -0.  0. -0.  0. -0.  1. -0. -0.  0. -0.]
 [ 0. -0. -0.  0. -0.  0. -0. -0.  1.  0.  0.  0.  0.  0. -0. -0.]
 [ 0. -0. -0.  0. -0. -0. -0.  0. -0.  0. -0.  0. -0.  1. -0. -0.]
 [ 0.  0.  0.  0.  0. -0.  0. -0.  0. -0. -0. -0. -0.  0.  1. 

### Sanity Check

In [76]:
# Create state <0001|
cir1 = UAnsatz(N)
cir1.basis_encoding(paddle.to_tensor([0, 0, 0, 1]))
v1 = cir1.run_state_vector()
print("Random state = \n", v1.numpy())

# Create state <0011|
cir2 = UAnsatz(N)
cir2.basis_encoding(paddle.to_tensor([0, 0, 1, 1]))
v2 = cir2.run_state_vector()
print("The target state = \n", v2.numpy())

# Sanity Check <0001|U^\dagger =? <0011|
print("\n=========================================")
print(v1.numpy()@U_compiled - v2.numpy())

Random state = 
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
The target state = 
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]

[0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]


YES! It works! Please feel free to check all the other transformations. 

## Discussion

The final trained circuit has a block depth of $D=40$ which looks very messy if we print it out... But all the parameters can be accessed through the following code if necessary:

In [78]:
# For more data visualization purpose

# theta_opt = net.theta.numpy()
# print("The trained parameter theta:\n", np.around(theta_opt, decimals=5))

We know the circuit architecture can always be improved. I didn't do that part because of the time constraint. Now, let's anser the question ``What would happen if you provided a different state"?

In [79]:
# Create state <1001|
cir3 = UAnsatz(N)
cir3.basis_encoding(paddle.to_tensor([1, 0, 0, 1]))
v3 = cir3.run_state_vector()
print("Random state = \n", v3.numpy())

# Sanity Check <0001|U^\dagger =? <0011|
print("\n=========================================")
print(v3.numpy()@U_compiled)

Random state = 
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]

[0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]


With this approach of circuit compiling, I already set all the other computation basis unchanged. Here is what would happen if we send a general state $|\Psi\rangle = \sum_{i=0}^{2^N-1} C_i|i\rangle$ through the circuit:

$$
U_\theta |\Psi\rangle = \sum_{|i\rangle \in S} C_i\cdot U_p |i\rangle + \sum_{|i\rangle \notin S} C_i\cdot I |i\rangle, \quad |C_i|^2=1.
\tag{7}
$$

## Other approaches -- Variational Quantum Classifier

- Assign a label to each input random state $S = \{(|\psi_0\rangle, y_0=0), (|\psi_1\rangle, y_1=1), (|\psi_2\rangle, y_2=2), (|\psi_3\rangle, y_3=3)\}$
- Train a Quantum Classifier $\tilde{y}_j = \mathcal{F}(|\psi_j\rangle, y_j)$ to recoginze the states correctly with labels by minimizing $\sum_j |\tilde{y}_j - \tilde{y}_j|^2$ or cross-entropy loss function
- Use those labels $\{\tilde{y}_j\}$ as an intermediate language (classical information) to control the generation fo those 4 targe states
